In [ ]:
import openmeteo_requests
import requests_cache
import pandas as pd
import numpy as np
import datetime as dt
import pytz
from retry_requests import retry
from metpy.calc import wind_components
from metpy.units import units


In [27]:
# Function to extract ordered hourly variables 
def get_ordered_hourly_variables(params):
    return params.get("hourly", [])

# Set up API client with caching and retries
cache_session = requests_cache.CachedSession('.cache', expire_after=3600)
retry_session = retry(cache_session, retries=5, backoff_factor=0.2)
openmeteo = openmeteo_requests.Client(session=retry_session)


def get_reading(url, latitude, longtitute, hourly, past_days):
  params = {
    "latitude": latitude,
    "longitude": longtitute,
    "hourly": hourly,
    "past_days": past_days,
    "forecast_days": 0
  }

  response = openmeteo.weather_api(url, params=params)[0]
  hourly = response.Hourly()
  hourly_data = {
    "date": pd.date_range(
        start=pd.to_datetime(hourly.Time(), unit="s", utc=True),
        end=pd.to_datetime(hourly.TimeEnd(), unit="s", utc=True),
        freq=pd.Timedelta(seconds=hourly.Interval()),
        inclusive="left"
    )
  }
  for i, var_name in enumerate(get_ordered_hourly_variables(params)):
    hourly_data[var_name] = hourly.Variables(i).ValuesAsNumpy()

  #Modify df
  hourly_df = pd.DataFrame(data=hourly_data)
  hourly_df['Date/Time'] = pd.to_datetime(hourly_df['date'], errors='coerce')
  hourly_df.set_index('Date/Time', inplace=True)
  hourly_df.drop(columns=['date'], inplace = True)

  return hourly_df

In [33]:
df = get_reading("https://api.open-meteo.com/v1/forecast",
            31.990534, 35.907787,
            ["temperature_2m", "dew_point_2m", "relative_humidity_2m",
           "pressure_msl", "wind_speed_10m", "wind_direction_10m"],
            5)


In [ ]:
df.rename(columns={
    "dew_point_2m": "Air Dew Point",
    "temperature_2m": "Air Temperature (OC)",
    "relative_humidity_2m": "Humidity %",
    "pressure_msl": "Atmospheric Pressure",
    "wind_speed_10m": "Wind Speed (MPS)",
    "wind_direction_10m": "Wind Direction (Degrees)"
}, inplace=True)

df.index = df.index.tz_convert(None)

u, v = wind_components(
    df['Wind Speed (MPS)'].values * units('m/s'),
    df['Wind Direction (Degrees)'].values * units.degree
)

df['Wind_U'] = u.magnitude  
df['Wind_V'] = v.magnitude
df.Wind_U = df.Wind_U.round(2)
df.Wind_V = df.Wind_V.round(2)

df.drop(columns=["Wind Speed (MPS)", "Wind Direction (Degrees)"], inplace=True)

In [36]:
df.head()


,Air Temperature (OC),Air Dew Point,Humidity %,Atmospheric Pressure,Wind_U,Wind_V
Date/Time,,,,,,
2025-04-23 00:00:00,21.690001,-4.438937,17.0,1014.400024,3.24,3.96
2025-04-23 01:00:00,20.990000,-5.806924,16.0,1013.799988,-1.08,5.04
2025-04-23 02:00:00,20.490000,-5.414185,17.0,1013.700012,-1.44,5.76
2025-04-23 03:00:00,20.389999,-4.740155,18.0,1013.599976,0.00,6.48
2025-04-23 04:00:00,21.240000,-2.631588,20.0,1013.599976,-1.08,6.84
